In [1]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pickle

In [1]:
#Setup Selenium
drive = webdriver.Chrome(ChromeDriverManager().install())
drive.maximize_window()
wait = WebDriverWait(drive, 10)

NameError: name 'webdriver' is not defined

In [7]:
#Getting informantion from each house websit:
def get_info():
    house_location = drive.find_element(By.XPATH, value = '//address[@class="post-address"]').text
    house_price = drive.find_element(By.XPATH, value = '//div[@class="item price"]').text
    house_area = drive.find_element(By.XPATH, value = '//div[@class="item acreage"]').text
    house_name = drive.find_element(By.XPATH, value = '//a[@style and @href and @title]').get_attribute('title')
    house_features = drive.find_element(By.XPATH, value = '//div[@class = "section-content"]//table[@class="table"]')
    house_features = house_features.find_elements(By.XPATH, value = '//tr')
    house_info = {}
    house_info['Location'] = house_location
    house_info['Price'] = house_price
    house_info['Name'] = house_name
    house_info['Area'] = house_area
    for index in range(len(house_features)):
        house_info['Feature' + str(index)] = house_features[index].text
    return house_info

In [8]:
#Scrapping
page = 1
house_infos = list()
while True:
    check = False
    try:
        drive.get('https://phongtro123.com/tinh-thanh/da-nang?page=' + str(page))
    except: break
    try:
        drive.find_element(By.XPATH, value = '//p[@style="color: red; text-align: center;"]')
    except: pass
    else:
        check = True
    if (check == True): break
    houses = drive.find_elements(By.XPATH, value = '//h3[@class="post-title"]//a[@style and @href]')
    if (houses == []): break
    for i in range(len(houses)):
        houses[i] = houses[i].get_attribute('href')
    for house in houses:
        drive.get(house)
        try:
            house_infos.append(get_info())
        except: continue
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('../Raw_Data/phongtro123.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
if (house_infos):
    with open('../Raw_Data/phongtro123.pickle','wb') as handle:
        pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
drive.close()

In [4]:
with open("../Raw_Data/phongtro123.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset.head()

,Location,Price,Name,Area,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10
0,"Địa chỉ: Đinh Đạt, Phường Phước Mỹ, Quận Sơn T...",2.8 triệu/tháng,"Cho thuê dài hạn phòng trọ khép kín, trần beto...",31m2,Mã tin: #622996,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin thường,"Ngày đăng: Thứ 7, 09:32 22/04/2023","Ngày hết hạn: Thứ 2, 09:32 22/05/2023",Liên hệ: Chi Hòa,Điện thoại: 0372932378,Zalo 0372932378,NaN
1,"Địa chỉ: Tôn Đức Thắng, Phường Hòa Minh, Quận ...",2 triệu/tháng,Cho thuê phòng trọ giá rẻ trên đường Tôn Đức T...,35m2,Mã tin: #622486,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin thường,"Ngày đăng: Thứ 3, 08:17 18/04/2023","Ngày hết hạn: Thứ 3, 08:17 25/04/2023",Liên hệ: WISE English - Tôn Đức Thắng,Điện thoại: 0326482674,Zalo 0326482674,NaN
2,"Địa chỉ: 96 Đường Trần Quý Khoách, Phường Hòa ...",2 triệu/tháng,"Cho thuê trọ, 96 Trần Quý Khoách, Liên Chiểu, ...",90m2,Mã tin: #620555,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin thường,"Ngày đăng: Thứ 2, 20:44 27/03/2023","Ngày hết hạn: Thứ 4, 20:44 26/04/2023",Liên hệ: phu pham,Điện thoại: 0777824725,Zalo 0777824725,NaN
3,"Địa chỉ: Đường Lê Văn Tâm, Phường Khuê Mỹ, Quậ...",2 triệu/tháng,"Cho thuê nhà trọ giá bình dân, khu vực an ninh...",25m2,Mã tin: #289447,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin thường,"Ngày đăng: Thứ 2, 16:51 30/03/2020","Ngày hết hạn: Thứ 6, 11:17 27/03/2020",Liên hệ: huan tran,Điện thoại: 0913414288,Zalo 0913414288,NaN
4,"Địa chỉ: 22A Phước Hòa, Phường Khuê Trung, Quậ...",1.5 triệu/tháng,"Cho thuê phòng trọ cho công nhân, sinh viên",15m2,Mã tin: #622056,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin miễn phí,"Ngày đăng: Thứ 4, 14:08 12/04/2023","Ngày hết hạn: Thứ 2, 14:08 17/04/2023",Liên hệ: Phan Thị Hoa,Điện thoại: 0903513345,Zalo 0903513345,NaN


In [3]:
dataset = dataset[['Location', 'Price', 'Area']]
dataset.head()

,Location,Price,Area
0,"Địa chỉ: Đinh Đạt, Phường Phước Mỹ, Quận Sơn T...",2.8 triệu/tháng,31m2
1,"Địa chỉ: Tôn Đức Thắng, Phường Hòa Minh, Quận ...",2 triệu/tháng,35m2
2,"Địa chỉ: 96 Đường Trần Quý Khoách, Phường Hòa ...",2 triệu/tháng,90m2
3,"Địa chỉ: Đường Lê Văn Tâm, Phường Khuê Mỹ, Quậ...",2 triệu/tháng,25m2
4,"Địa chỉ: 22A Phước Hòa, Phường Khuê Trung, Quậ...",1.5 triệu/tháng,15m2


In [ ]:
dataset.to_csv('../Raw_Data/phongtro123.csv')